In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

In [2]:
import pandas as pd
import numpy as np
from joblib import load
from global_vs_local_surrogate import print_report

/Users/francesco/anaconda3/envs/tesi/lib/python3.6/site-packages/deap/tools/_hypervolume/pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)
Using TensorFlow backend.


In [3]:
path = parentdir + "/agnostic_explainers/"

In [4]:
blackboxes = ["resnet", "knn", "simplecnn"]
methods = ["geneticp", "random"]
dataset_names = ["esr", "har", "cbf", "poc"]
metrics = ["fid_g","fid_l"]

In [6]:
df_dict = dict()
for blackbox in blackboxes:
    for method in methods:
        df_dict[(blackbox, method)] = []
columns = list(df_dict.keys())
rows = [(dataset, metric) for dataset in dataset_names for metric in metrics]
folders = os.listdir(path)
folders.remove('.DS_Store')
folders.remove('noscrape')
folders = sorted(folders)
for i, folder in enumerate(folders):
    folderlist = folder.split("_")[:3]
    print("dataset:", folderlist[0])
    print("method:", folderlist[1])
    print("bb:", folderlist[2])
    if "missing" in folder:
        results_df = pd.read_csv(parentdir + "/agnostic_explainers/" + folder + "/" + "results_df.csv", sep = ";")
        results = print_report(results_df, only_global = True)
        df_dict[(folderlist[2], folderlist[1])].append(('%.3f' % (results["fid_g"])).replace('0.', '.') if results["fid_g"] < 1.0 else '1.00')
        df_dict[(folderlist[2], folderlist[1])].append(results["fid_l"])
        
    else:
        results_df = pd.read_csv(parentdir + "/agnostic_explainers/" + folder + "/" + "results_df.csv", sep = ";")
        results = print_report(results_df)
        df_dict[(folderlist[2], folderlist[1])].append(('%.3f' % (results["fid_g"])).replace('0.', '.') if results["fid_g"] < 1.0 else '1.00')
        df_dict[(folderlist[2], folderlist[1])].append(('%.3f' % (results["fid_l"])).replace('0.', '.') if results["fid_l"] < 1.0 else '1.00')
    print()

dataset: EpilepticSeizureRecognition
method: geneticp
bb: knn
reconstruction fidelity:  0.98
local fidelity:  0.94
global fidelity:  0.92

dataset: EpilepticSeizureRecognition
method: geneticp
bb: resnet
reconstruction fidelity:  0.98
local fidelity:  1.0
global fidelity:  0.96

dataset: EpilepticSeizureRecognition
method: geneticp
bb: simplecnn
reconstruction fidelity:  0.98
local fidelity:  1.0
global fidelity:  0.96

dataset: EpilepticSeizureRecognition
method: random
bb: knn
reconstruction fidelity:  0.98
local fidelity:  0.82
global fidelity:  0.92

dataset: EpilepticSeizureRecognition
method: random
bb: resnet
reconstruction fidelity:  0.98
local fidelity:  0.5
global fidelity:  0.96

dataset: EpilepticSeizureRecognition
method: random
bb: simplecnn
reconstruction fidelity:  nan
local fidelity:  nan
global fidelity:  0.96

dataset: HARDataset
method: geneticp
bb: knn
reconstruction fidelity:  0.98
local fidelity:  0.98
global fidelity:  0.38

dataset: HARDataset
method: geneticp


In [7]:
df_dict

{('resnet', 'geneticp'): ['.960',
  '1.00',
  '.420',
  '.980',
  '1.00',
  '1.00',
  '.660',
  '.500'],
 ('resnet', 'random'): ['.960',
  '.500',
  '.420',
  '.460',
  '1.00',
  '1.00',
  '.660',
  '.400'],
 ('knn', 'geneticp'): ['.920',
  '.940',
  '.380',
  '.980',
  '1.00',
  '1.00',
  '.740',
  '.940'],
 ('knn', 'random'): ['.920',
  '.820',
  '.380',
  '.400',
  '1.00',
  '1.00',
  '.740',
  '.440'],
 ('simplecnn', 'geneticp'): ['.960',
  '1.00',
  '.400',
  '.860',
  '1.00',
  '.889',
  '.660',
  '.960'],
 ('simplecnn', 'random'): ['.960',
  nan,
  '.400',
  '.440',
  '1.00',
  '.917',
  '.660',
  '.400']}

In [8]:
df=pd.DataFrame(df_dict, index=pd.MultiIndex.from_tuples(rows))

In [16]:
df=df[[(   'resnet', 'geneticp'),(   'resnet',   'random'),
    ('simplecnn', 'geneticp'),('simplecnn',   'random'),
    (      'knn', 'geneticp'),(      'knn',   'random')]].loc[["cbf","esr","har","poc"]]

In [17]:
df

resnet        simplecnn             knn       
          geneticp random  geneticp random geneticp random
esr fid_g     .960   .960      .960   .960     .920   .920
    fid_l     1.00   .500      1.00    NaN     .940   .820
har fid_g     .420   .420      .400   .400     .380   .380
    fid_l     .980   .460      .860   .440     .980   .400
cbf fid_g     1.00   1.00      1.00   1.00     1.00   1.00
    fid_l     1.00   1.00      .889   .917     1.00   1.00
poc fid_g     .660   .660      .660   .660     .740   .740
    fid_l     .500   .400      .960   .400     .940   .440

In [18]:
column_format = "|c|c||"
for i in range(len(df.columns)):
    column_format += "c"
column_format += "|"
latex = df.to_latex(index = True, 
                    bold_rows = True,
                    column_format = column_format,
                    multicolumn_format = "c",
                    multirow = True)
latex = latex.replace("toprule", "hline")
latex = latex = latex.replace("midrule", "hline")
latex = latex.replace("bottomrule", "hline")
print(latex)
print()
print()
print()

\begin{tabular}{|c|c||cccccc|}
\hline
    &       & \multicolumn{2}{c}{resnet} & \multicolumn{2}{c}{simplecnn} & \multicolumn{2}{c}{knn} \\
    &       & geneticp & random &  geneticp & random & geneticp & random \\
\hline
\multirow{2}{*}{\textbf{esr}} & \textbf{fid\_g} &     .960 &   .960 &      .960 &   .960 &     .920 &   .920 \\
    & \textbf{fid\_l} &     1.00 &   .500 &      1.00 &    NaN &     .940 &   .820 \\
\cline{1-8}
\multirow{2}{*}{\textbf{har}} & \textbf{fid\_g} &     .420 &   .420 &      .400 &   .400 &     .380 &   .380 \\
    & \textbf{fid\_l} &     .980 &   .460 &      .860 &   .440 &     .980 &   .400 \\
\cline{1-8}
\multirow{2}{*}{\textbf{cbf}} & \textbf{fid\_g} &     1.00 &   1.00 &      1.00 &   1.00 &     1.00 &   1.00 \\
    & \textbf{fid\_l} &     1.00 &   1.00 &      .889 &   .917 &     1.00 &   1.00 \\
\cline{1-8}
\multirow{2}{*}{\textbf{poc}} & \textbf{fid\_g} &     .660 &   .660 &      .660 &   .660 &     .740 &   .740 \\
    & \textbf{fid\_l} &     .500 & 

# CILINDER BELL FUNNEL

### RESNET

#### GENETICP

In [12]:
folder_name = "cbf_resnet_20191223_122259"
results_df = pd.read_csv(parentdir + "/agnostic_explainers/" + folder_name + "/" + "results_df.csv", sep = ";")
print_report(results_df)

local fidelity:  1.0
global fidelity:  1.0
reconstruction fidelity:  1.0


#### RANDOM

In [13]:
folder_name = "cbf_random_resnet_20200105_191824"
results_df = pd.read_csv(parentdir + "/agnostic_explainers/" + folder_name + "/" + "results_df.csv", sep = ";")
print_report(results_df)

local fidelity:  1.0
global fidelity:  1.0
reconstruction fidelity:  1.0


### CNN

#### GENETICP

In [20]:
folder_name = "cbf_simplecnn_20200109_175704"
results_df = pd.read_csv(parentdir + "/agnostic_explainers/" + folder_name + "/" + "results_df.csv", sep = ";")
print_report(results_df)

local fidelity:  0.8888888888888888
global fidelity:  1.0
reconstruction fidelity:  0.8888888888888888


#### RANDOM

In [21]:
folder_name = "cbf_random_simplecnn_20200109_152246"
results_df = pd.read_csv(parentdir + "/agnostic_explainers/" + folder_name + "/" + "results_df.csv", sep = ";")
print_report(results_df)

local fidelity:  0.9166666666666666
global fidelity:  1.0
reconstruction fidelity:  0.8888888888888888


# EPILEPTIC SEIZURE RECOGNITION

### RESNET

#### GENETICP

In [14]:
folder_name = "EpilepticSeizureRecognition_20200106_192815"
results_df = pd.read_csv(parentdir + "/agnostic_explainers/" + folder_name + "/" + "results_df.csv", sep = ";")
print_report(results_df)

local fidelity:  1.0
global fidelity:  0.96
reconstruction fidelity:  0.98


#### RANDOM

In [15]:
folder_name = "EpilepticSeizureRecognition_random_20200107_010425"
results_df = pd.read_csv(parentdir + "/agnostic_explainers/" + folder_name + "/" + "results_df.csv", sep = ";")
print_report(results_df)

local fidelity:  0.5
global fidelity:  0.96
reconstruction fidelity:  0.98


### CNN

#### GENETICP

In [22]:
folder_name = "EpilepticSeizureRecognition_simplecnn_20200109_180414"
results_df = pd.read_csv(parentdir + "/agnostic_explainers/" + folder_name + "/" + "results_df.csv", sep = ";")
print_report(results_df)

local fidelity:  1.0
global fidelity:  0.96
reconstruction fidelity:  0.98


#### RANDOM

In [ ]:
# MISSING

# HUMAN ACTIVITY RECOGNITION

### RESNET

#### GENETICP

In [16]:
folder_name = "HARDataset_20191223_212314"
results_df = pd.read_csv(parentdir + "/agnostic_explainers/" + folder_name + "/" + "results_df.csv", sep = ";")
print_report(results_df)

local fidelity:  0.98
global fidelity:  0.42
reconstruction fidelity:  0.98


#### RANDOM

In [17]:
folder_name = "HARDataset_random_20200105_213622"
results_df = pd.read_csv(parentdir + "/agnostic_explainers/" + folder_name + "/" + "results_df.csv", sep = ";")
print_report(results_df)

local fidelity:  0.46
global fidelity:  0.42
reconstruction fidelity:  0.98


### CNN

#### GENETICP

In [23]:
folder_name = "HARDataset_simplecnn_20200109_180238"
results_df = pd.read_csv(parentdir + "/agnostic_explainers/" + folder_name + "/" + "results_df.csv", sep = ";")
print_report(results_df)

local fidelity:  0.86
global fidelity:  0.4
reconstruction fidelity:  0.86


#### RANDOM

In [ ]:
# MISSING

# PHALANGES

In [ ]:
# labels errate in caso di grafici. da sostituire con None in caso

### RESNET

#### GENETICP

In [18]:
folder_name = "phalanges_20191223_160501"
results_df = pd.read_csv(parentdir + "/agnostic_explainers/" + folder_name + "/" + "results_df.csv", sep = ";")
print_report(results_df)

local fidelity:  0.5
global fidelity:  0.66
reconstruction fidelity:  0.98


#### RANDOM

In [19]:
folder_name = "phalanges_random_20200105_195552"
results_df = pd.read_csv(parentdir + "/agnostic_explainers/" + folder_name + "/" + "results_df.csv", sep = ";")
print_report(results_df)

local fidelity:  0.4
global fidelity:  0.66
reconstruction fidelity:  0.98


### CNN

#### GENETICP

In [24]:
folder_name = "phalanges_simplecnn_20200109_181310"
results_df = pd.read_csv(parentdir + "/agnostic_explainers/" + folder_name + "/" + "results_df.csv", sep = ";")
print_report(results_df)

local fidelity:  0.96
global fidelity:  0.66
reconstruction fidelity:  1.0


#### RANDOM

In [25]:
folder_name = "phalanges_random_simplecnn_20200111_104034"
results_df = pd.read_csv(parentdir + "/agnostic_explainers/" + folder_name + "/" + "results_df.csv", sep = ";")
print_report(results_df)

local fidelity:  0.4
global fidelity:  0.66
reconstruction fidelity:  1.0
